# GUN IMAGE
1-304 NO GUN
305-608 GUN


In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import  img_to_array
import re
import numpy as np
# Image manipulations and arranging data
import os
from PIL import Image
# import theano
# theano.config.optimizer="None"
#Sklearn to modify the data
from sklearn.model_selection import train_test_split
# os.chdir("provide path")
# input image dimensions
from keras.layers import LSTM

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
m,n = 160,120
path2='/content/drive/MyDrive/Colab Notebooks/DM/data/Classification'

In [8]:
classes=os.listdir(path2)
x=[]
y=[]
count = 0
for fol in classes:
  print (fol)
  imgfiles=os.listdir(path2 + '/' + fol);
  for img in imgfiles:
    try:
      im=Image.open(path2+'/'+fol+'/'+img);
      im=im.convert(mode='RGB')
      imrs=im.resize((m,n))
      imrs=img_to_array(imrs)/255;
      imrs=imrs.transpose(2,0,1);
      imrs=imrs.reshape(3,m,n);
      x.append(imrs)
      y.append(count)
    except:
      pass
  count += 1

knife
pistol


In [9]:
x=np.array(x);
y=np.array(y);

In [13]:
x.shape

(6324, 160, 120, 3)

In [14]:
x=x.reshape(6324,160, 120,3)

In [15]:
batch_size=32
nb_classes=len(classes)
nb_epoch=20
nb_filters=128
nb_pool=2
nb_conv=3

x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2,random_state=4)

uniques, id_train=np.unique(y_train,return_inverse=True)
Y_train=np_utils.to_categorical(id_train,nb_classes)
uniques, id_test=np.unique(y_test,return_inverse=True)
Y_test=np_utils.to_categorical(id_test,nb_classes)

In [39]:
model= Sequential()
model.add(Conv2D(nb_filters,(3,3),padding='same',input_shape=(160,120,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(Conv2D(int(nb_filters/2),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(Dropout(0.2))
model.add(Conv2D(int(nb_filters/4),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(Conv2D(int(nb_filters/8),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(Dropout(0.2));
model.add(Flatten());
model.add(Dense(128));
model.add(Dropout(0.2))
model.add(Dense(2,Activation('softmax')));
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['acc',f1_m,precision_m, recall_m])

In [82]:
model= Sequential()
model.add(Conv2D(nb_filters,(3,3),padding='same',input_shape=(160,120,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(Conv2D(int(nb_filters/2),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));


model.add(Dropout(0.2))

model.add(Conv2D(int(nb_filters/4),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));

model.add(Conv2D(int(nb_filters/8),(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)));
model.add(TimeDistributed(LSTM(64, activation='relu')))
#model.add((LSTM(64, activation='relu')))

model.add(Dropout(0.2));

model.add(Flatten());
model.add(Dense(128));
model.add(Dropout(0.2))
model.add(Dense(2,Activation('softmax')));
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['acc',f1_m,precision_m, recall_m])

In [83]:
model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_111 (Conv2D)          (None, 160, 120, 128)     3584      
_________________________________________________________________
max_pooling2d_110 (MaxPoolin (None, 80, 60, 128)       0         
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 78, 58, 64)        73792     
_________________________________________________________________
max_pooling2d_111 (MaxPoolin (None, 39, 29, 64)        0         
_________________________________________________________________
dropout_62 (Dropout)         (None, 39, 29, 64)        0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 37, 27, 32)        18464     
_________________________________________________________________
max_pooling2d_112 (MaxPoolin (None, 18, 13, 32)      

In [84]:
nb_epoch=60
batch_size=16
model.fit(x_train,Y_train,batch_size=batch_size,epochs=nb_epoch,verbose=1,validation_data=(x_test, Y_test))
model.save("gun_detect.h5",overwrite=True)

Epoch 1/60
317/317 [==============================] - 9s 30ms/step - loss: 0.6477 - acc: 0.6270 - f1_m: 0.6262 - precision_m: 0.6262 - recall_m: 0.6262 - val_loss: 0.6449 - val_acc: 0.6245 - val_f1_m: 0.6172 - val_precision_m: 0.6172 - val_recall_m: 0.6172
Epoch 2/60
317/317 [==============================] - 9s 28ms/step - loss: 0.6266 - acc: 0.6276 - f1_m: 0.6285 - precision_m: 0.6285 - recall_m: 0.6285 - val_loss: 0.6238 - val_acc: 0.6245 - val_f1_m: 0.6289 - val_precision_m: 0.6289 - val_recall_m: 0.6289
Epoch 3/60
317/317 [==============================] - 9s 28ms/step - loss: 0.6069 - acc: 0.6327 - f1_m: 0.6328 - precision_m: 0.6328 - recall_m: 0.6328 - val_loss: 0.6401 - val_acc: 0.6024 - val_f1_m: 0.6070 - val_precision_m: 0.6070 - val_recall_m: 0.6070
Epoch 4/60
317/317 [==============================] - 9s 28ms/step - loss: 0.5920 - acc: 0.6448 - f1_m: 0.6448 - precision_m: 0.6448 - recall_m: 0.6448 - val_loss: 0.5831 - val_acc: 0.6632 - val_f1_m: 0.6672 - val_precision_m: 0.

In [85]:
path2='/content/drive/MyDrive/Colab Notebooks/DM/data/Test'

In [86]:
files=os.listdir(path2);
x=[]
y=[]
for i in files:
  im = Image.open(path2+'/' + i);
  im=im.convert(mode='RGB')
  imrs=im.resize((m,n))
  imrs=img_to_array(imrs)/255;
  imrs=imrs.transpose(2,0,1);
  imrs=imrs.reshape(m,n,3);
  x.append(imrs)
  num = re.sub(r'\D', "", i)
  if int(num)<305:
    y.append(0)
  else:
    y.append(1)
x=np.array(x);
y=np.array(y);
#print (model.summary())

In [87]:
Y_sample=np_utils.to_categorical(y)

In [88]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x,Y_sample,verbose=1)

19/19 [==============================] - 0s 13ms/step - loss: 0.5023 - acc: 0.8333 - f1_m: 0.8337 - precision_m: 0.8337 - recall_m: 0.8337
